In [2]:
from helper.splitter import sentence_splitter
import re
import pandas as pd
from nltk.tokenize import wordpunct_tokenize, word_tokenize, TweetTokenizer

In [3]:
#test
#sentence = "~kk RT @GoddesofKebab lg😭😭😭 500rb😭🤲🏻 10juta bapak.aku internet! -ok temen\"ku refund-nya baik-baik baik/buruk nge-print (ads) (rumah kita) era 4.0 ngasih? tender :)) proyek, I'm Bob's I'll apps ovo...ke vendor abal2. #mantapkali https://towardsdatascience.com/pos-tagging-using-crfs-ea430c5fb78b"
#stc = ' Wis \'lah...tidur satu\' "Gw dapet bedanya” ibu" 500rb😭🤲🏻 “halah tinggal, ngeprint. sendiri apa bedanya” terburu-buru tok..hahaha 2.3 !ok iki..haha mari..." .ki...,, ovo...ke vendor abal2. 10% 10juta siang/malam bapak2 &ok makan.minum bener² ibu\" -__-  (´･ω･`) he!! apa??? and -_- SIRAMPOG .mari...\" 14.07.2021 #instaNews Senin 14-07-2021 21/10/19 [telah] {terjadi} +angin kencang, di-print termasuk Sirampog. Kondisi (terkini) Sirampog, @100.000 aik-aik 12+ .. *info *tunggu* update dari ?kita ya! lur .. Do\'a kan yg…  https://t.co/7Dl8UV2QZU'
#stc = '":v :V ;v lg😭😭😭 500rb😭🤲🏻  -_- -__- dapet bola aja jarang! Susah! Sekalinya dapet, pelanggaran! MAU KAMU APAA??" “halah tinggal ngeprint sendiri apa bedanya” 🤣🤣🤣🤣 lemot "@kecepoood: XL labil donlod munggah mudun 😩"'
#stc = '(´･ω･`) \(^▿^)/ bener² baju\'e I\'m “halah may*ora df~~~ ~~~fsf tinggal, iki..haha !dfs &sdf !fsd mari. dsfg, ,sdf .ki ...,, ovo...ke (sendiri) (sdf aja) ngeprint. apa bedanya”  ~ff -fg sdf~ sfs- 4.3 iki..haha !dfs &sdf !fsd mari. dsfg, ,sdf .ki ...,, ovo...ke (sendiri) (sdf aja) $10 Rp10.000,00 11-23-22 12/12/32 15.00 @10.1000 #sdfs @sfsdf https://pynative.com/python-regex-compile/  -= STASIUN CITAYAM =- " @sdfsd: beda :\") ;)) 🤣🤣🤣🤣'
sentence = 'hati-hati, ji-as kk-ds. baju. 3x 1,9 12.45 3-4 asda-asd! satu+dua'
print(sentence_splitter(sentence))
print(word_tokenize(sentence))
print(wordpunct_tokenize(sentence))
#sentence.split()
#stc.split()

#PR: 1-3 not tokenized correctly, asda-asdasd, sddfs+sdfds,

['hati-hati', ',', 'ji-as', 'kk-ds', '.', 'baju', '.', '3x', '1,9', '12.45', '3-4', 'asda-asd', '!', 'satu', '+', 'dua']
['hati-hati', ',', 'ji-as', 'kk-ds', '.', 'baju', '.', '3x', '1,9', '12.45', '3-4', 'asda-asd', '!', 'satu+dua']
['hati', '-', 'hati', ',', 'ji', '-', 'as', 'kk', '-', 'ds', '.', 'baju', '.', '3x', '1', ',', '9', '12', '.', '45', '3', '-', '4', 'asda', '-', 'asd', '!', 'satu', '+', 'dua']


In [6]:
#target_file = '../dataset/new-tagged-500.tsv'
#source_file = '../dataset/filtered-tweet-500.xlsx'
#target_file = '../dataset/new-tagged-300.tsv'
#source_file = '../dataset/data-150322.xlsx'
#target_file = '../dataset/new-tagged-1000.tsv'
#source_file = '../dataset/data-210322.xlsx'
target_file = '../raw dataset/all-tagged-test-data.tsv'
source_file = '../raw dataset/data-050422-test.xlsx'

In [7]:
all_data = pd.read_excel(source_file)
tweets = all_data['tweet'].head(600)

In [ ]:
#all_data = pd.read_excel(source_file, header=None)
#tweets = all_data[0]
#len(all_data)
#tweets = all_data['tweets']

In [8]:
len(tweets)

600

In [9]:
# Indonesian dictionary
id_dict_file = pd.read_csv('../dict/indonesian_words_final.csv')
id_dict = id_dict_file.word

In [10]:
# named entity dictionary
ne_dict = pd.read_csv('../dict/NE-list.txt', header=None)

In [11]:
# Javanese dictionary
jv_dict_file = pd.read_csv('../dict/unknown_words_java.csv')
jv_dict = jv_dict_file.word

In [12]:
# English dictionary
en_dict_file = pd.read_csv('../dict/english_words_final.csv')
en_dict = en_dict_file.word

In [13]:
# Mix ID-EN dictionary
mix_id_en_file = pd.read_csv('../dict/mixed_words_final.csv')
mix_id_en_dict = mix_id_en_file.word

In [14]:
# Mix JV-EN dictionary
mix_jv_en_dict = pd.read_csv('../dict/jv-en.txt', header=None)

In [15]:
# Mix ID-JV dictionary
mix_id_jv_dict = pd.read_csv('../dict/id-jv.txt', header=None)

In [16]:
def tagger(target_file, input_data):
	tweets = input_data
	with open(target_file, 'a', encoding='utf-8') as f:
		for tw in tweets:
			tokens = sentence_splitter(tw)
			for token in tokens:
				tlower = token.lower()
				if token.startswith('#') or token.startswith('@') or token.endswith('%') or token.isnumeric():
					f.write(token + '\t' + 'O' + '\n')
				elif re.match(r'https?:\/\/.*[\r\n]*',token) or re.match(r'[\W]', token):
					f.write(token + '\t' + 'O' + '\n')
				elif tlower in ne_dict.values:
					f.write(token + '\t' + 'NE' + '\n')
				elif tlower in id_dict.values:
					f.write(token + '\t' + 'ID' + '\n')
				elif tlower in en_dict.values:
					f.write(token + '\t' + 'EN' + '\n')
				elif tlower in jv_dict.values:
					f.write(token + '\t' + 'JV' + '\n')
				elif tlower in mix_id_en_dict.values:
					f.write(token + '\t' + 'MIX-ID-EN' + '\n')
				elif tlower in mix_id_jv_dict.values:
					f.write(token + '\t' + 'MIX-ID-JV' + '\n')
				elif tlower in mix_jv_en_dict.values:
					f.write(token + '\t' + 'MIX-JV-EN' + '\n')
				else:
					f.write(token + '\t' + '\n')
			f.write('\n')

In [17]:
tagger(target_file=target_file, input_data=tweets)

UnicodeEncodeError: 'charmap' codec can't encode character '\u23f0' in position 0: character maps to <undefined>